# Practica de Clasificacion. Parte 2

Volveremos a utilizar la tabla con datos de pacientes con diagnostico positivo y negativo de diabetes pero esta vez separaremos la tabla de entrenamiento en dos, entrenamiento y validacion, para ver como distintos parametros elegidos para el algoritmo de clasificacion pueden afectar los resultados.

In [ ]:
import pandas as pd

Esta vez cargaremos las tablas ya preprocesadas

In [ ]:
df_train = pd.read_csv('data/diabetes/diabetes_train_procesado.csv')
df_test = pd.read_csv('data/diabetes/diabetes_test_procesado.csv')

In [ ]:
X_train = df_train.loc[:, df_train.columns != "diabetes"]
y_train = df_train.loc[:, "diabetes"]

X_test = df_test.loc[:, df_test.columns != "diabetes"]
y_test = df_test.loc[:, "diabetes"]

## Dividir la tabla de entrenamiento

Dividir la tabla de entrenamiento en entrenamiento y validacion

In [ ]:
from sklearn.model_selection import train_test_split
X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, train_size=0.70, random_state=42)

## Probar distintos parametros y medir

Probaremos ahora distintos parametros para el algoritmo de Arbol de Decision y calcularemos la cantidad de aciertos para las tablas de entrenamiento y validacion

In [ ]:
from sklearn.metrics import accuracy_score

def calcular_accuracy_train_val(clf, X_train, y_train, X_val, y_val):
    clf.fit(X_train,y_train)
    y_train_pred = clf.predict(X_train)
    print("Entrenamiento accuracy:",accuracy_score(y_train, y_train_pred))
    y_val_pred = clf.predict(X_val)
    print("Validacion accuracy:",accuracy_score(y_val, y_val_pred))
    return clf

In [ ]:
from sklearn.tree import DecisionTreeClassifier

Podemos ver todos los parametros de este clasificador en la pagina oficial de sklearn: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [ ]:
arbol1 = DecisionTreeClassifier(random_state=42)
clf = calcular_accuracy_train_val(arbol1, X_train2, y_train2, X_val, y_val)

In [ ]:
arbol2 = DecisionTreeClassifier(criterion="entropy", splitter="best", random_state=42)
clf = calcular_accuracy_train_val(arbol2, X_train2, y_train2, X_val, y_val)

In [ ]:
arbol3 = DecisionTreeClassifier(criterion="entropy", splitter="random",random_state=42)
clf = calcular_accuracy_train_val(arbol3, X_train2, y_train2, X_val, y_val)

Finalmente seleccionamos la combinacion de parametros que nos ha dado el mejor resultado y calculamos las metricas de Entrenamiento y Prueba. En este caso el mejor arbol es el que usa el parametro criterion="entropy" y ahora utilizamos las tablas de Entrenamiento y Prueba originales.

In [ ]:
clf = DecisionTreeClassifier(criterion="entropy",random_state=42)
clf.fit(X_train,y_train)
y_train_pred = clf.predict(X_train)
print("Entrenamiento accuracy:",accuracy_score(y_train, y_train_pred))
y_test_pred = clf.predict(X_test)
print("Prueba accuracy:",accuracy_score(y_test, y_test_pred))

## Validacion Cruzada y Busqueda de grilla

Ahora utilizaremos las funciones de [Validacion Cruzada](https://scikit-learn.org/stable/modules/cross_validation.html) y [Busqueda de Grilla](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) que provee sklearn para automatizar este proceso y obtener los mejores hiperparametros para cada clasificador. La funcion de busqueda de grilla me permite pasarle los parametros que queremos probar y automaticamente probara todos los modelos, hara cross validation para evaluarlos y nos devolvera cuales han sido los mejores parametros:

In [ ]:
mis_parametros_a_probar = { 
    'splitter' : ['best','random'],
    'criterion' :['gini', 'entropy', 'log_loss'],
    'random_state' : [42]
}

In [ ]:
from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(
    estimator=DecisionTreeClassifier(), 
    param_grid=mis_parametros_a_probar, 
    cv=5)
cv.fit(X_train, y_train)
cv.best_params_

Finalmente obtenemos el que ha dado el mejor resultado a traves del parametro **best_estimator_**

In [ ]:
clf = cv.best_estimator_
clf.fit(X_train,y_train)
y_train_pred = clf.predict(X_train)
print("Entrenamiento accuracy:",accuracy_score(y_train, y_train_pred))
y_test_pred = clf.predict(X_test)
print("Prueba accuracy:",accuracy_score(y_test, y_test_pred))

## Ejercicio 1:
    
Repetir el proceso para el clasificador de [regresion logistica](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) utilizando la funcion GridSearchCV()

In [ ]:
# Escribir aqui la solucion




In [ ]:
#@title Solucion Ejercicio 1 {display-mode:"form"}

mis_parametros_a_probar = { 
    'penalty' : ['l2',None],
    'max_iter' :[300],
    'random_state' : [42]
}

from sklearn.linear_model import LogisticRegression
cv = GridSearchCV(
    estimator=LogisticRegression(), 
    param_grid=mis_parametros_a_probar, 
    cv=5)
cv.fit(X_train, y_train)
print("Los mejores parametros son:", cv.best_params_)

clf = cv.best_estimator_
clf.fit(X_train,y_train)
y_train_pred = clf.predict(X_train)
print("Entrenamiento accuracy:",accuracy_score(y_train, y_train_pred))
y_test_pred = clf.predict(X_test)
print("Prueba accuracy:",accuracy_score(y_test, y_test_pred))

## Ejercicio 2:
    
Repetir el proceso para el clasificador de [HistGradientBoosting](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html) utilizando la funcion GridSearchCV()

In [ ]:
# Escribir aqui la solucion




In [ ]:
#@title Solucion Ejercicio 2 {display-mode:"form"}

# Notemos aqui que ya se empieza a tardar mucho cuando los parametros a probar
# son muchos y el algoritmo es lento 

mis_parametros_a_probar = { 
    'learning_rate' : [0.05, 0.1],
    'min_samples_leaf' : [20, 30],
    'random_state' : [42]
}

from sklearn.ensemble import HistGradientBoostingClassifier
cv = GridSearchCV(
    estimator=HistGradientBoostingClassifier(), 
    param_grid=mis_parametros_a_probar, 
    cv=3)
cv.fit(X_train, y_train)
print("Los mejores parametros son:", cv.best_params_)

clf = cv.best_estimator_
clf.fit(X_train,y_train)
y_train_pred = clf.predict(X_train)
print("Entrenamiento accuracy:",accuracy_score(y_train, y_train_pred))
y_test_pred = clf.predict(X_test)
print("Prueba accuracy:",accuracy_score(y_test, y_test_pred))

# Fin: [Volver al contenido del curso](https://www.freecodingtour.com/cursos/espanol/datascience/datascience.html)